In [1]:
############ LIBRARIES ###############

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import csv
import tsfel

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=RuntimeWarning)
simplefilter(action='ignore', category=FutureWarning)

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

%matplotlib inline

# variavel que evita NaN nos resultados
epslon = 0.00005

def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_pred - y_true)/np.sum(y_true))
  else:
   return 100*(np.sum(y_pred - y_true)/(np.sum(y_true)+ epslon))  

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

#função para normalização
def znorm(x):
  if np.std(x)!=0:
    x_znorm = (x - np.mean(x)) / np.std(x)
  else:
    x_znorm = (x - np.mean(x)) / (np.std(x) + epslon) 
   
  return x_znorm

#função para desnormatização
def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds


C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
#### DEFs ####

def extract_estado(file_name):
    # Split the file name by underscores
    parts = file_name.split('_')
    # Extract the name between underscores
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    # List all files in the folder
    files = os.listdir(folder_path)
    # Iterate through each file
    for file_name in files:
        # Check if it's a CSV file
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Open the CSV file and read the data
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                # Assuming the first row contains headers
                headers = next(reader)
                # Extract estado from file name and append to estados list
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados


############################################################


# Função para extração de Features utilizando Tsfel
def rolling_window_featureTsfel(series, window):
  data_out=[]
  data_out2=pd.DataFrame(data_out)
  data_out3=[]
  for i in range(len(series)-window): #Itera sobre a diferença entre o tamanho da série e o tamanho da janela
    example = np.array(series[i:i+window])
    cfg = tsfel.get_features_by_domain() # Recupera um conjunto de características pré-definidas para extração
    new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características
    new_elements_values_reshaped = np.squeeze(new_elements.values) # Remove dimensão unitária, transformando de (1, 140) para (140,)
    data_featuredf =  pd.DataFrame(new_elements_values_reshaped.T)
    data_out = (data_featuredf.T)
    data_out = data_out.apply(znorm, axis=1)
    data_out3 = pd.DataFrame(data_out)
    data_out2 = pd.concat([data_out2, data_out3], ignore_index=True)
  return data_out2 #retorna a criação de um DataFrame com o resultado dos novos dados e as características extraídas


#########################################################


def recursive_multistep_forecasting_TsFel(y_train, model, horizon):
    seriec = y_train.tail(horizon)
    preds = []
    for i in range(horizon):
        example = np.array(seriec)
        cfg = tsfel.get_features_by_domain() # Recupera um conjunto de características pré-definidas para extração
        new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características
        new_elements_values_reshaped = np.squeeze(new_elements.values) # Remove dimensão unitária, transformando de (1, 140) para (140,)
        data_featuredf=  pd.DataFrame(new_elements_values_reshaped.T)
        data_out = data_featuredf.T
        data_out3 = pd.DataFrame(data_out)
        data_out3.fillna(0, inplace=True)
        data_out3.replace([float('inf'), -float('inf')], 0, inplace=True)
        pred = model.predict(data_out3)
        pred = np.nan_to_num(pred, nan=0)
        preds.append(pred)
        series2 = seriec[1:]
        seriec = np.append(series2, pred)
    return preds



#########################################################



In [5]:
############## TsFel SAVE ##############

import pickle

horizon = 12
window = 12
features = 'TsFel'

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:

        # carregamento do arquivo

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        series = df['m3']

        TimeStamp = df['timestamp'].tail(398) 
        TimeStamp.reset_index(drop=True, inplace=True) 

        outTsFel = rolling_window_featureTsfel(series, window) 

        outTsFel = outTsFel.fillna(0) 
 
        outTsFel.insert(0, 'timestamp', TimeStamp) 

        folder_name = 'TsFEL'
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        outTsFel.to_csv(f'{folder_name}/FEAT_TsFel_{product}_{window}_{estado}.csv', index=False)


*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_19044\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_19044\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


...


*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_19044\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_19044\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características


KeyboardInterrupt: 

In [6]:
############## TsFel TEST ##############

import pickle

horizon = 12
window = 12
features = 'TsFel'


## Depende da janela e da Feature
if window == 12:
    FeatRegister = 386 
elif window ==24:
    FeatRegister = 374
else:
    FeatRegister = 362

#################################################
product = 'etanolhidratado'
estado = 'ac'
#################################################

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
# for product in products:
#     folder_path = f'./uf/{product}/'
#     # Read the CSV files and extract estado names
#     estados = read_csv_files(folder_path)
#     for estado in estados:

# carregamento do arquivo

df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
series = df['m3']

TimeStamp = df['timestamp'].tail(398) 
TimeStamp.reset_index(drop=True, inplace=True) 

outTsFel = rolling_window_featureTsfel(series, window) 

outTsFel = outTsFel.fillna(0) 
outTsFel.replace([float('inf'), -float('inf')], 0, inplace=True)

###############################################

y_norm = rolling_window(series,window)

outTsFel['y'] = df['m3'].tail(FeatRegister).reset_index(drop=True)

# outTsFel['y'] = y_norm.iloc[:, [-1]]

X_train, X_test, y_train, y_test = train_test_split(outTsFel, horizon)

############## Regressores ##################

regr1 = LinearRegression()
regr2 = KNeighborsRegressor(n_neighbors = 3)
regr3 = XGBRegressor()
regr4 = SVR(kernel='rbf')
regr5 = RandomForestRegressor()
regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)

regr1.fit(X_train, y_train)
predictions1a = recursive_multistep_forecasting_TsFel(y_train, regr1, horizon)
predictions1a = [a.flatten()[0] for a in predictions1a]

regr2.fit(X_train, y_train)
predictions2a = recursive_multistep_forecasting_TsFel(y_train, regr2, horizon)
predictions2a = [a.flatten()[0] for a in predictions2a]

regr3.fit(X_train, y_train)
predictions3a = recursive_multistep_forecasting_TsFel(y_train, regr3, horizon)
predictions3a = [a.flatten()[0] for a in predictions3a]

regr4.fit(X_train, y_train)
predictions4a = recursive_multistep_forecasting_TsFel(y_train, regr4, horizon)
predictions4a = [a.flatten()[0] for a in predictions4a]

regr5.fit(X_train, y_train)
predictions5a = recursive_multistep_forecasting_TsFel(y_train, regr5, horizon)
predictions5a = [a.flatten()[0] for a in predictions5a]

regr6.fit(X_train, y_train)
predictions6a = recursive_multistep_forecasting_TsFel(y_train, regr6, horizon)
predictions6a = [a.flatten()[0] for a in predictions6a]

Valores_Reais = series.tail(horizon)
Valores_Reais = Valores_Reais.reset_index(drop=True)

mape_result1 = mape(Valores_Reais, predictions1a)
mape_result2 = mape(Valores_Reais, predictions2a)
mape_result3 = mape(Valores_Reais, predictions3a)
mape_result4 = mape(Valores_Reais, predictions4a)
mape_result5 = mape(Valores_Reais, predictions5a)
mape_result6 = mape(Valores_Reais, predictions6a)

pbe_result1 = pbe(Valores_Reais, predictions1a)
pbe_result2 = pbe(Valores_Reais, predictions2a)
pbe_result3 = pbe(Valores_Reais, predictions3a)
pbe_result4 = pbe(Valores_Reais, predictions4a)
pbe_result5 = pbe(Valores_Reais, predictions5a)
pbe_result6 = pbe(Valores_Reais, predictions6a)

pocid_result1 = pocid(Valores_Reais, predictions1a)
pocid_result2 = pocid(Valores_Reais, predictions2a)
pocid_result3 = pocid(Valores_Reais, predictions3a)
pocid_result4 = pocid(Valores_Reais, predictions4a)
pocid_result5 = pocid(Valores_Reais, predictions5a)
pocid_result6 = pocid(Valores_Reais, predictions6a)

predictions1b = ', '.join(f"{item:f}" for item in predictions1a)
predictions2b = ', '.join(f"{item:f}" for item in predictions2a)
predictions3b = ', '.join(f"{item:f}" for item in predictions3a)
predictions4b = ', '.join(f"{item:f}" for item in predictions4a)
predictions5b = ', '.join(f"{item:f}" for item in predictions5a)
predictions6b = ', '.join(f"{item:f}" for item in predictions6a)
    
rows_data = [
    [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
    [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
    [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
    [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
    [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
    [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],          
]

# Salva Modelos
folder_path = f"./00-MODELS_UF_MENSAL-TsFel/"
os.makedirs(folder_path, exist_ok=True)

with open(os.path.join(folder_path, f'TsFel_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
    pickle.dump(regr2, fd1)
with open(os.path.join(folder_path, f'TsFel_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
    pickle.dump(regr2, fd2)
with open(os.path.join(folder_path, f'TsFel_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
    pickle.dump(regr3, fd3)
with open(os.path.join(folder_path, f'TsFel_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
    pickle.dump(regr4, fd4)
with open(os.path.join(folder_path, f'TsFel_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
    pickle.dump(regr5, fd5)
with open(os.path.join(folder_path, f'TsFel_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
    pickle.dump(regr2, fd6)

# CSV Output VALORES REAIS
with open(f'TsFel_{window}_output.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    for row_data in rows_data:
        writer.writerow(row_data)


*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_19044\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_19044\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


...


*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_19044\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_19044\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características


KeyboardInterrupt: 

In [62]:
############## TsFel AUTO ##############

import pickle

horizon = 12
window = 12
features = 'TsFel'


## Depende da janela e da Feature
if window == 12:
    FeatRegister = 386 
elif window ==24:
    FeatRegister = 374
else:
    FeatRegister = 362

#################################################
# product = 'etanolhidratado'
# estado = 'ac'
#################################################

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    for estado in estados:

        # carregamento do arquivo

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        series = df['m3']

        TimeStamp = df['timestamp'].tail(398) 
        TimeStamp.reset_index(drop=True, inplace=True) 

        outTsFel = rolling_window_featureTsfel(series, window) 

        outTsFel = outTsFel.fillna(0) 
        outTsFel.replace([float('inf'), -float('inf')], 0, inplace=True)

        ###############################################

        y_norm = rolling_window(series,window)

        outTsFel['y'] = df['m3'].tail(FeatRegister).reset_index(drop=True)

        # outTsFel['y'] = y_norm.iloc[:, [-1]]

        X_train, X_test, y_train, y_test = train_test_split(outTsFel, horizon)

        ############## Regressores ##################

        regr1 = LinearRegression()
        regr2 = KNeighborsRegressor(n_neighbors = 3)
        regr3 = XGBRegressor()
        regr4 = SVR(kernel='rbf')
        regr5 = RandomForestRegressor()
        regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)

        regr1.fit(X_train, y_train)
        predictions1a = recursive_multistep_forecasting_TsFel(y_train, regr1, horizon)
        predictions1a = [a.flatten()[0] for a in predictions1a]

        regr2.fit(X_train, y_train)
        predictions2a = recursive_multistep_forecasting_TsFel(y_train, regr2, horizon)
        predictions2a = [a.flatten()[0] for a in predictions2a]

        regr3.fit(X_train, y_train)
        predictions3a = recursive_multistep_forecasting_TsFel(y_train, regr3, horizon)
        predictions3a = [a.flatten()[0] for a in predictions3a]

        regr4.fit(X_train, y_train)
        predictions4a = recursive_multistep_forecasting_TsFel(y_train, regr4, horizon)
        predictions4a = [a.flatten()[0] for a in predictions4a]

        regr5.fit(X_train, y_train)
        predictions5a = recursive_multistep_forecasting_TsFel(y_train, regr5, horizon)
        predictions5a = [a.flatten()[0] for a in predictions5a]

        regr6.fit(X_train, y_train)
        predictions6a = recursive_multistep_forecasting_TsFel(y_train, regr6, horizon)
        predictions6a = [a.flatten()[0] for a in predictions6a]

        Valores_Reais = series.tail(horizon)
        Valores_Reais = Valores_Reais.reset_index(drop=True)

        mape_result1 = mape(Valores_Reais, predictions1a)
        mape_result2 = mape(Valores_Reais, predictions2a)
        mape_result3 = mape(Valores_Reais, predictions3a)
        mape_result4 = mape(Valores_Reais, predictions4a)
        mape_result5 = mape(Valores_Reais, predictions5a)
        mape_result6 = mape(Valores_Reais, predictions6a)

        pbe_result1 = pbe(Valores_Reais, predictions1a)
        pbe_result2 = pbe(Valores_Reais, predictions2a)
        pbe_result3 = pbe(Valores_Reais, predictions3a)
        pbe_result4 = pbe(Valores_Reais, predictions4a)
        pbe_result5 = pbe(Valores_Reais, predictions5a)
        pbe_result6 = pbe(Valores_Reais, predictions6a)

        pocid_result1 = pocid(Valores_Reais, predictions1a)
        pocid_result2 = pocid(Valores_Reais, predictions2a)
        pocid_result3 = pocid(Valores_Reais, predictions3a)
        pocid_result4 = pocid(Valores_Reais, predictions4a)
        pocid_result5 = pocid(Valores_Reais, predictions5a)
        pocid_result6 = pocid(Valores_Reais, predictions6a)

        predictions1b = ', '.join(f"{item:f}" for item in predictions1a)
        predictions2b = ', '.join(f"{item:f}" for item in predictions2a)
        predictions3b = ', '.join(f"{item:f}" for item in predictions3a)
        predictions4b = ', '.join(f"{item:f}" for item in predictions4a)
        predictions5b = ', '.join(f"{item:f}" for item in predictions5a)
        predictions6b = ', '.join(f"{item:f}" for item in predictions6a)
            
        rows_data = [
            [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
            [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
            [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
            [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
            [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
            [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],          
        ]

        # Salva Modelos
        folder_path = f"./00-MODELS_UF_MENSAL-TsFel/"
        os.makedirs(folder_path, exist_ok=True)

        with open(os.path.join(folder_path, f'TsFel_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
            pickle.dump(regr2, fd1)
        with open(os.path.join(folder_path, f'TsFel_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
            pickle.dump(regr2, fd2)
        with open(os.path.join(folder_path, f'TsFel_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
            pickle.dump(regr3, fd3)
        with open(os.path.join(folder_path, f'TsFel_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
            pickle.dump(regr4, fd4)
        with open(os.path.join(folder_path, f'TsFel_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
            pickle.dump(regr5, fd5)
        with open(os.path.join(folder_path, f'TsFel_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
            pickle.dump(regr2, fd6)

        # CSV Output VALORES REAIS
        with open(f'TsFel_{window}_output.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            for row_data in rows_data:
                writer.writerow(row_data)


*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_18124\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_18124\279256520.py:42: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


...


*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_18124\279256520.py:61: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características



*** Feature extraction finished ***
*** Feature extraction started ***


C:\Users\jonas\AppData\Local\Temp\ipykernel_18124\279256520.py:61: UserWarning: Using default sampling frequency set in configuration file.
  new_elements = tsfel.time_series_features_extractor(cfg, example) # Extração das características


KeyboardInterrupt: 